This notebook shows how to access the truth trajectory information for each event.

This information is extracted from the simulation and is available to help quantify the accuracy and precision of our reconstruction and analysis algorithms.

We also show how to project that information into the wireplane images. This provides and overview of the different "times" that we deal with in the experiment.

In [ ]:
import os,sys
import plotly as pl
import plotly.graph_objects as go
%load_ext autoreload
%autoreload 2

In [ ]:
# on trex, for some reason, need to load ROOT in separate cell before loading icdl modules
import ROOT as rt

In [ ]:
from larlite import larlite
from larcv import larcv
from ublarcvapp import ublarcvapp
from larlite import larutil
import lardly

Below we specify the location of the file we want to use.

For a list of example files, refer to this [list](https://github.com/NuTufts/icdl/wiki/Locations-of-Tutorial-Files).

In [ ]:
# Specify location of the file we want to open

# For list of tutorial files check out []
#inputfile = "/tutorial_files/merged_dlreco_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root"
inputfile = "/home/twongjirad/working/data/mcc9_v13_bnbnue_corsika/merged_dlreco_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root"

In [ ]:
# Use ROOT to dump the data of the file
# you'll see a lot of TTree objects. 
# These are essentially tables of data with a specific schema, 
#   with each row of data consisting of one "event".

# For this tutorial we'll need the following TTree(s)
# * mctrack_mcreco_tree
# * mcshower_mcreco_tree

rfile = rt.TFile(inputfile,"open")
rfile.ls()

We do not access the data (i.e. Trees) directly). The ROOT trees store data in the form of serialized instances of custom C++ classes! In otherwords, for each event, copies of C++ classes are turned into a binary string, storing the values of its data members. Unpacking this involves de-serializing the class data and giving back a set of C++ class instances for each event. We have special classes of our own that interfaces with ROOT's IO functions to help us do this.

There is two we need to use.

The first is `larlite::storage_manager` which interfaces data products from larlite, which is a clone of the data products from larsoft. (Why do we use larlite and not larsoft directly? larlite does not require the large number of dependencies that are hard to build on systems that are not on Fermilab.)

The second is `larcv::IOManager` which provides the interface to the images, which we'll use at the end of the notebook.

In [ ]:
# We use the larcv IOManager class to load the trees for us.

# Initialize an storage_manager instance that will read out input file
ioll = larlite.storage_manager( larlite.storage_manager.kREAD )
ioll.add_in_filename( inputfile )
ioll.open()

# Initialize an IOManager instance that will read out input file
iolcv = larcv.IOManager( larcv.IOManager.kREAD, "larcv", larcv.IOManager.kTickBackward )
iolcv.add_in_file( inputfile )
iolcv.reverse_all_products()
iolcv.initialize()



In [ ]:
# Now we load an event

ENTRY_NUM = 0
ioll.go_to(ENTRY_NUM)
iolcv.read_entry(ENTRY_NUM)


In [ ]:
# Get the containers holding the mctrack and mcshower objects

event_mctrack  = ioll.get_data( larlite.data.kMCTrack,  "mcreco" )
event_mcshower = ioll.get_data( larlite.data.kMCShower, "mcreco" )

print("Number of mctracks: ",event_mctrack.size())
print("Number of mcshowers: ",event_mcshower.size())

In [ ]:
# Because we'll need to orient the information within the geometry of the MicroBooNE detector,
# we need to activate it.
larutil.LArUtilConfig.SetDetector( larlite.geo.kMicroBooNE )


In [ ]:
# Before we vizualize this information, we can dump out this information
# We can use a utility that will try to arrange the list of tracks and showers
# by their mother-daughter relationship

# warning, its a bit of a mess

mcpg = ublarcvapp.mctools.MCPixelPGraph()
mcpg.buildgraphonly( ioll )
mcpg.printGraph(0, False)

In [ ]:
import lardly
from lardly.detectoroutline import get_tpc_boundary_plot

tpclines = get_tpc_boundary_plot()


In [ ]:
## Run this block if you want to draw just the TPC plot that we retrieved

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

plot_layout = {
    "title": "",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

fig = go.Figure( data=[tpclines], layout=plot_layout )
fig.show()

Below we'll plot the Truth track trajectories. 

In the lardly code, the following color scheme is used:

```
default_pid_colors = {2212:'rgb(153,55,255)', # protons                                       
                      13:'rgb(255,0,0)', # muons                               
                      -13:'rgb(255,0,0)', # muons                          
                      211:'rgb(255,128,255)',# pions                       
                      -211:'rgb(255,128,255)',# pions                      
                      0:'rgb(0,0,0)'# other                                  
                      }
```

What you should notice is that all the tracks are inside the TPC.
This is because only true trajectory information is saved inside the TPC
(exception is the creation origin) which is kept somewhere in the mctrack object.

The code that takes a container of mctrack objects and returns respective plot objects is

```
lardly.data.visualize_larlite_event_mctrack( event_mctrack, apply_t0_offset=False )
```

Note that `apply_t0_offset=False`.

In [ ]:
# make plot objects for the track objects

plot_mctracks = lardly.data.visualize_larlite_event_mctrack( event_mctrack, apply_t0_offset=False )



axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

plot_layout = {
    "title": "",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

fig = go.Figure( data=[tpclines]+plot_mctracks, layout=plot_layout )
fig.show()


Now we remake the track objects, but set `apply_t0_offset=False`.

What you'll see is that the tracks are now outside the TPC.
This has to do with the way we measure and reconstruct
the time of trajectories in the detector.


In [ ]:
# make plot objects for the track objects

plot_mctracks = lardly.data.visualize_larlite_event_mctrack( event_mctrack, apply_t0_offset=True )


axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

plot_layout = {
    "title": "",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 2, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

fig = go.Figure( data=[tpclines]+plot_mctracks, layout=plot_layout )
fig.show()


# DAQ and Timing in the TPC

To understand the timing in the detector, we need to describe how the data is saved.

There are electronics that continually stores a rolling queue of voltage versus time measurements.

The time between samples (sometimes called ticks) is determined by the electronics clock used. For MicroBooNE, the data acquisition system  (or DAQ) used to record the voltage versus time on all the wires saves a sample every 0.5 microseconds, i.e. 0.5 microseconds per tick.
There is some amount of stored values in the electronics. Values are replaced in a First-In-First-Out (FIFO) manner.

Every so often, a signal is sent to the electronics to save the stored values to disk.
This signal is called the trigger.
We use a number of different triggers. 
The primary one is a signal that comes a little bit ahead of when the neutrino beam will arrive.

The saved data is arranged in time order and a window of data is defined. 
This is how the data is arranged from MicroBooNE.
The tick that occured at the same time of the trigger is positioned to always be at `tick=3200`.
The range of samples stored goes from 3200 ticks before the trigger and 6400 ticks after the trigger to a total window size of 9600 ticks.
In order to save space, MicroBooNE in the first process of data processing removes the first 2400 ticks and 1152 ticks at the end for a truncated 6448 tick window.

In our code library, we retain this 2400-8448 tick range as the "coordinate" for the time axis of the image data.
In otherwords, we say the first "row" of the image is at tick=2400 and the last row ends at tick=8448.

Ionization left behind by charged particles in the detector arrive at the wireplanes and makes a signal in the electronics and thereby the image.  However, before the ionization arrives at the wireplane, the ionization must first drift from the location it was created to the wireplanes. Assuming a constant velocity (a first-order approximation we often use), the between the particle actually traveled the detector and the time the ionization shows up in the image is `(distance from wireplanes)/(drift velocity)`. The drift velocity depends on the electric field used in the detector. For MicroBooNE the drift velocity was measured to be around 0.111 cm per microseconds.

The original time of the trajectory (in others the real time) is known as the `t0` of the trajectory.
The `t0_offset` is the drift time or `(distance from wireplanes)/(drift velocity)`.  So the time that the signal appears on the wire plane is `t = t0 + t0_offset`.

So how can we find out this offset? Well, we actually are able to measure the original time of a trajectory.
In addition to ionization, most trajectories will produce a sizable flash of scintillation light that is seen in the optical detectors.  The time of the event can be measured potentially by the light is at the nanoseconds level.  However, associating flashes of light in the optical system to the charged trajectories is not an easy task and is not something we can do at the start.  You first have to decide what signals or pixels in an image are due trajectories occuring nearly at the same time.  Then you need to match that spatial pattern of ionization with the amount and spatial pattern of light signals in the optical detectors.  As a result, the `t0_offset` for a trajectory cannot be subtracted out until the middle or later portions of the reconstrucion.

Therefore, we often set the time of a trajectory as simply the relative to the original `tick=3200` event trigger.
And given that time offset, we use the const drift velocity to position the track in the detector. 

This is all to explain why when we plot the truth trajectories using the `apply_t0_offset` flag, the tracks are outside the TPC. This is because their timing (and equivalently) their position is all relative to `tick=3200`.
Plotting the true trajectories this way is useful for comparing against reconstruction trajectories before a `t0_offset` can be applied.


In [ ]:
# get detector outlines

PLANE = 0 # options are 0,1, or 2 for the U,V,Y plane
plane_image = adc_v.at(PLANE)
plane_plot = lardly.data.visualize_larcv_image2d( plane_image, reverse_ticks=True )

# plotly figure
fig = go.Figure( data=[plane_plot] )
fig.show()

## If opening the tutorial file, merged_dlreco_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root,
## the image should be of a Charged-Current Electron Neutrino Interaction
## If on the U-plane, the neutrino interaction vertex is around (1650,5328)

For more information on the classes, please follow the links below.

Used in this tutorial:

* [larcv::IOManager](https://github.com/LArbys/LArCV/blob/develop/larcv/core/DataFormat/IOManager.h)
* [larcv::Image2D](https://github.com/LArbys/LArCV/blob/develop/larcv/core/DataFormat/Image2D
.h)
* [plotly heatmap](https://plotly.com/python/heatmaps/)